In [1]:
import pickle
from tqdm import tqdm
import sys

# TO CHANGE
BASEDIR = "../"
# TO CHNAGE
sys.path.insert(0, BASEDIR)

from src.utils.data_structs import TripletCreator, NodeCreator, Relation, NODES_TYPES_MAP, RELATIONS_TYPES_MAP
from src.neo4j_functions import Neo4jConnection, Neo4jConnectionConfig

NEO4J_URL ="bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PWD = "password"
GRAPH_DB_NAME = 'testdb'

/home/dzigen/Desktop/PersonalAI/pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
graph_db = Neo4jConnection(Neo4jConnectionConfig(uri=NEO4J_URL, user=NEO4J_USER, pwd=NEO4J_PWD, db_name=GRAPH_DB_NAME))

In [3]:
raw_triplets = list(graph_db.execute_query("MATCH (n1)-[rel]->(n2) RETURN n1, rel, n2"))
formated_triplets = []
for raw_triplet in tqdm(raw_triplets):
    start_node = NodeCreator.create(id=raw_triplet['n1'].element_id, name=str(raw_triplet['n1']['name']), 
                                        type=NODES_TYPES_MAP[list(raw_triplet['n1'].labels)[0]],
                                        prop=dict(raw_triplet['n1']))
    end_node = NodeCreator.create(id=raw_triplet['n2'].element_id, name=str(raw_triplet['n2']['name']), 
                                        type=NODES_TYPES_MAP[list(raw_triplet['n2'].labels)[0]],
                                        prop=dict(raw_triplet['n2']))
    relation = Relation(id=raw_triplet['rel'].element_id, name=str(raw_triplet['rel']['name']), 
                            type=RELATIONS_TYPES_MAP[raw_triplet['rel'].type], 
                            prop=dict(raw_triplet['rel']))
    
    triplet = TripletCreator.create(start_node, relation, end_node, add_stringified_triplet=False)
    formated_triplets.append(triplet)

print("formated_triplets amount: ", len(formated_triplets))

100%|██████████| 10355/10355 [00:00<00:00, 28799.22it/s]

formated_triplets amount:  10355


In [4]:
with open(f'../data/pickled_graphs/{GRAPH_DB_NAME}.pickle', 'wb') as f:
    pickle.dump(formated_triplets, f)